## Demonstration: Automating the Detection of Trends via Python

This note book demonstrates how Python can be used to automate the process of checking for trends in time series data. It assumes that data files that include time series data for multiple products exist in a sub-folder of the working directory. The following code block imports several libraries that we will use.

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os

The following code block allows a user to define a subfolder that includes the data files with the time series data for the various products and generate plots of each time series. The plots are written to a sub-folder of the current working directory named *images*. The code assumes that the each data files has columns:
- `Period` that specifies the period, and 
- `Value` that specifes the time series value in each period.

Of course, you would not want to inspect the images to detect tie series trends because this would be rather time-consuming when the number of products is large. However, we will use these plots to check the correctness of our computational approach.

In [2]:
# Identify the sub-folder that contains the data files
sub_folder = 'data_small'

# Loop through all files in the sub-folder
for file in os.listdir(sub_folder +'/'):
    
    # Update the filename variable to include the
    # sub-folder and file name
    filename = sub_folder +'/' + file
    
    # Use Pandas to read the current csv
    # file into a DataFrame
    data = pd.read_csv(filename)
        
    # Plot the time series
    fig, ax = plt.subplots(1,1)
    ax.plot(data['Period'], data['Value'])
    ax.set_xlabel('Period')
    ax.set_ylabel('Value')
    ax.set_ylim(bottom = data['Value'].min() - (data['Value'].max()-data['Value'].min()),
               top = data['Value'].max() + (data['Value'].max()-data['Value'].min()))
    ax.set_title(file[:-4] + ' Values')
    
    # Save the plot in the images sub-folder
    plt.savefig('images/' + file[:-4] + '.pdf')
    
    # Close the plot
    plt.close()

For each data file in the user-specified sub-folder, the following code block:

1. imports the data into a Pandas `DataFrame` object,
2. fits a first-degree polynomial (i.e., a line) to the current data, and 
3. stores the current product, fit intercept, and fit slope in separate lists.    

In [3]:
products = []
intercepts = []    
slopes = []

# Identify the sub-folder that contains the data files
sub_folder = 'data_small'

# Loop through all files in the sub-folder
for file in os.listdir(sub_folder +'/'):
    
    # Update the filename variable to include the
    # sub-folder and file name
    filename = sub_folder +'/' + file
    
    # Use Pandas to read the current csv
    # file into a DataFrame
    data = pd.read_csv(filename)
    
    # Add the current product name to the products list
    products.append(file[:-4])
    
    # Fit a first-degree polynomial (i.e., a line)
    # to the current time series
    est = np.polyfit(data['Period'], data['Value'], 1)
    
    # Append the fit slope to the slopes list
    slopes.append(est[0])
    
    # Append the fit intercept to the intercepts list
    intercepts.append(est[1])

The following code block:
    
1. uses the lists constructed in the previous code block to create a dictionary,
2. uses the dictionary to define a Pandas `DataFrame`,
3. adds a column to the `DataFrame` that captures the absolute percent change as $abs(Slope/Intercept)$,
4. sorts the `DataFrame` by absolute percent change, from high to low, and
5. writes the `DataFrame` object to an Excel workbook.

In [4]:
# Create a dictionary from the created lists
my_dict = {'Product': products,
           'Intercept': intercepts,
           'Slope': slopes}

# Use the dictionary to create a Pandas DataFrame
# object that is stored in the variable summary
summary = pd.DataFrame.from_dict(my_dict)

# Calculate the absolute percent change for each row of the 
# summary object
summary['Percent_Change'] = abs(summary['Slope']/summary['Intercept'])

# Sort the data by percent change, from high to low
summary = summary.sort_values(by = 'Percent_Change', ascending = False)

# Write the summary object to an Excel workbook
summary = summary.to_excel('Summary.xlsx', index = False)